In [1]:
from farm_haystack.reader.adaptive_model import FARMReader
from farm_haystack.retriever.tfidf import TfidfRetriever
from farm_haystack import Finder
from farm_haystack.indexing.io import write_documents_to_db, fetch_archive_from_http
from farm_haystack.indexing.cleaning import clean_wiki_text
from farm_haystack.utils import print_answers

11/11/2019 18:43:25 - INFO - root -   Using localhost sqlite as the database backend. as Database not configured. Add a qa_config.py file in the Python path with DATABASE_URL set.Continuing with the default sqlite on localhost.
I1111 18:43:25.533496 140652943304512 file_utils.py:39] PyTorch version 1.3.0 available.
I1111 18:43:25.611577 140652943304512 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [2]:
# Init a database (default: sqllite)
from farm_haystack.database import db
db.create_all()

# Let's first get some documents that we want to query
# Here: Wikipedia articles for Game of Thrones

doc_dir = "data/article_txt_got"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"

fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# We supply a function to clean the docs and write them afterwards to our DB
write_documents_to_db(document_dir=doc_dir, clean_func=clean_wiki_text)

I1111 18:43:25.834958 140652943304512 io.py:46] Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip to `data/article_txt_got`
100%|██████████| 1167348/1167348 [00:00<00:00, 13662984.26B/s]
I1111 18:43:26.548232 140652943304512 io.py:31] Wrote 517 to DB


In [3]:
# A retriever identifies the k most promising chunks of text that might contain the answer for our question
# Retrievers use some simple but fast algorithm, here: TF-IDF

retriever = TfidfRetriever()

I1111 18:43:26.589386 140652943304512 tfidf.py:68] Found 517 candidates in DB


In [ ]:
# A reader scans the text chunks in detail and extracts the k best answers
# Reader use more powerful but slower deep learning models, here: a BERT QA model trained via FARM on Squad 2.0
reader = FARMReader(model_dir="../FARM/saved_models/bert-english-qa-large")

In [ ]:
# The Finder sticks together retriever and retriever in a pipeline to answer our actual questions 

finder = Finder(reader, retriever)

In [ ]:
# Voilá! Ask a question!

#prediction = finder.get_answers(question="Who created the Dothraki vocabulary?", top_k_reader=5)
#prediction = finder.get_answers(question="Who is the sister of Sansa?", top_k_reader=5)
prediction = finder.get_answers(question="Who is the father of Arya Stark?", top_k_reader=3, top_k_retriever=5)

In [ ]:
print_answers(prediction, details="minimal")